In [15]:
# create a model that uses the fenchel-young loss and trains on 200 images 
# the images are permuted in 9 tiles and in every iteration a new permutation is generated.
# we use a deep encoder with fixed parameters, adam optimizer, preprocessing before tiling  

import tensorflow as tf

In [16]:
from ast import Pass
import platform
import cv2
import numpy as np
import os
from pathlib import Path

number_of_images = -1

root = os.getcwd() # Don't change this
data_dirname = '/data_test/plantvillage/' # Change as you like 
p = Path(root + data_dirname)
p.mkdir(exist_ok=True) 
if platform.system()=='Darwin':
  root = os.getcwd() # Don't change this
  data_dirname = '/data_test/plantvillage/' # Change as you like 
  p = Path(root + data_dirname)
  p.mkdir(exist_ok=True) 
else:
  #p = Path("C:/Users/mwels/Documents/Uni/11. Semester/Deep learning in visual recognition/Plant_leave_diseases_dataset_without_augmentation")
  #p.mkdir(exist_ok=True)
  pass



classes = [
  'Apple___Apple_scab',
  'Apple___healthy',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  "all"
  ]

if "all" in classes:
  classes = os.listdir(p)

for c in classes:
  print(c,end=" ")
  filelist = [x for x in (p/c).iterdir() if x.is_file()]
  for f in filelist:
    img = cv2.imread(str(f))
    if img is None:
      print(f'Failed to open {f}. Deleting file')
      os.remove(str(f))


filelist = filelist[:number_of_images]
print(len(filelist))

Apple___Apple_scab Apple___Black_rot Apple___Cedar_apple_rust Apple___healthy 1644


In [17]:
len(filelist)

1644

In [18]:
tilex = 3
number_of_tiles = tilex**2
number_of_permutations = 1000
target_siz = (224,224,3)
tile_size = target_siz[0]//tilex
sfmax = True

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

conv_base = MobileNet(weights='imagenet',
                      include_top=False,
                      input_shape=target_siz)

In [19]:
from keras.layers import Dense, Concatenate, Input, Flatten, Lambda, GlobalAveragePooling2D
from keras.models import Model
import os


#tiles = Input((number_of_tiles,tile_size,tile_size,3))
tiles = Input(target_siz)

inputs = {}
layers = {}
embedds = {}

shared_conv = conv_base 

# for i in range(number_of_tiles):
#     #inputs[f'tiles{i}'] = Input((tile_size,tile_size,3))
#     #layers[f'tile{i}'] = Lambda(lambda x: x[:,i,:,:,:])(tiles)

#     #layers[f'deep_layers{i}'] = shared_conv(inputs[f'tiles{i}'])
#     layers[f'deep_layers{i}'] = shared_conv(layers[f'tile{i}'])
#     embedds[f'embedd{i}'] = Flatten()(layers[f'deep_layers{i}'])
#concatonation = Concatenate(axis=1)(list(embedds.values()))

concatonation = shared_conv(tiles)
concatonation = GlobalAveragePooling2D()(concatonation)

out = Dense(number_of_tiles*10, activation="relu", kernel_initializer='he_normal')(concatonation)
if sfmax:
    out = Dense(number_of_permutations, activation="softmax", kernel_initializer='he_normal')(out)
else:
    out = Dense(number_of_tiles, kernel_initializer='he_normal')(out)
out = Flatten()(out)

model = Model(inputs=tiles, outputs=out)
#model = Model(inputs=list(inputs.values()), outputs=out)

In [20]:
tf.keras.utils.plot_model(model, 
    show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [21]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 90)                92250     
                                                                 
 dense_3 (Dense)             (None, 1000)              91000     
                                                                 
 flatten_1 (Flatten)         (None, 1000)              0   

In [22]:
total_num_layers = len(model.layers)
num_base_layers = len(conv_base.layers)
print(f"Total number of layers is {total_num_layers}")
print(f"Number of pretrained base layers is {num_base_layers}")

for layer in model.layers[:3]:
    layer.trainable=False
for layer in model.layers[3:]:
    layer.trainable=True

Total number of layers is 6
Number of pretrained base layers is 86


In [23]:
sys.path.append('src')

In [24]:
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.PermOneHotDataGen import *
from src.model_tools import *
from src.permutation_tools import *

x_train, x_test = train_test_split(filelist)

train_generator = PermOneHotDataGen(x_train,
                                    batch_size=8,
                                    tilenumberx=tilex, 
                                    shuffle_permutations=True)

validation_generator = PermOneHotDataGen(x_test,
                                        batch_size=8,
                                        tilenumberx=tilex,
                                        shuffle_permutations=True)

train_generator_sm = PermOneHotDataGen(x_train,
                                    batch_size=1,
                                    tilenumberx=tilex, max_perms=number_of_permutations)

validation_generator_sm = PermOneHotDataGen(x_test,
                                        batch_size=1,
                                        tilenumberx=tilex, max_perms=number_of_permutations)

train_stitch_sm = PermOneHotDataGen(x_train,
                                    batch_size=16,
                                    tilenumberx=tilex,
                                    shuffle_permutations=True,
                                    max_perms=number_of_permutations,
                                    target_size=target_siz, 
                                    stitched=True,
                                    one_hot_encoding=True)

validation_stitch_sm = PermOneHotDataGen(x_test,
                                        batch_size=len(x_test),
                                        tilenumberx=tilex, 
                                        max_perms=number_of_permutations, 
                                        target_size=target_siz, 
                                        stitched=True,
                                        one_hot_encoding=True)


optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',
)

if sfmax:
    model.compile(optimizer=optimizer,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy'])
else:
    model.compile(
        optimizer=optimizer,
        loss=RankingLoss(),
        metrics=[ProjectedRanksAccuracy(), PartialRanksAccuracy()])


In [25]:
print(train_generator.next()[1].shape)
print(train_generator_sm.next()[1].shape)
print(validation_generator.next()[1].shape)
print(validation_generator_sm.next()[1].shape)
print(train_generator.next()[0].shape)
print(train_generator_sm.next()[0].shape)
print(validation_generator.next()[0].shape)
print(validation_generator_sm.next()[0].shape)

(8, 9)
(1, 9)
(8, 9)
(1, 9)
(8, 9, 85, 85, 3)
(1, 9, 85, 85, 3)
(8, 9, 85, 85, 3)
(1, 9, 85, 85, 3)


In [26]:
ProjectedRanksAccuracy().update_state(validation_generator.next()[1], validation_generator.next()[1])

In [27]:
import numpy as np 
from tensorflow import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output

class PlotLearning(tf.keras.callbacks.Callback):
    """
    Callback to plot the learning curves of the model during training.
    """
    def on_train_begin(self, logs={}):
        self.metrics = {}
        for metric in logs:
            self.metrics[metric] = []
            

    def on_epoch_end(self, epoch, logs={}):
        # Storing metrics
        for metric in logs:
            if metric in self.metrics:
                self.metrics[metric].append(logs.get(metric))
            else:
                self.metrics[metric] = [logs.get(metric)]
        
        # Plotting
        metrics = [x for x in logs if 'val' not in x]
        
        f, axs = plt.subplots(1, len(metrics), figsize=(15,5))
        clear_output(wait=True)

        for i, metric in enumerate(metrics):
            axs[i].plot(range(1, epoch + 2), 
                        self.metrics[metric], 
                        label=metric)
            if logs['val_' + metric]:
                axs[i].plot(range(1, epoch + 2), 
                            self.metrics['val_' + metric], 
                            label='val_' + metric)
                
            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

In [28]:
nb_epochs = 10

model.fit(train_stitch_sm,
          epochs = nb_epochs,
          validation_data=validation_stitch_sm,
          verbose=1)

Epoch 1/10
78/78 [==============================] - 19s 231ms/step - loss: 6.9745 - accuracy: 0.0016 - val_loss: 6.9092 - val_accuracy: 0.0000e+00
Epoch 2/10
78/78 [==============================] - 17s 220ms/step - loss: 6.8361 - accuracy: 0.0041 - val_loss: 6.9179 - val_accuracy: 0.0000e+00
Epoch 3/10
78/78 [==============================] - 17s 213ms/step - loss: 6.6325 - accuracy: 0.0081 - val_loss: 7.0591 - val_accuracy: 0.0024
Epoch 4/10
78/78 [==============================] - 17s 212ms/step - loss: 6.3104 - accuracy: 0.0154 - val_loss: 7.2956 - val_accuracy: 0.0024
Epoch 5/10
78/78 [==============================] - 17s 212ms/step - loss: 5.9992 - accuracy: 0.0203 - val_loss: 7.6331 - val_accuracy: 0.0000e+00
Epoch 6/10
78/78 [==============================] - 17s 212ms/step - loss: 5.6799 - accuracy: 0.0333 - val_loss: 8.0010 - val_accuracy: 0.0024
Epoch 7/10
78/78 [==============================] - 17s 213ms/step - loss: 5.3856 - accuracy: 0.0543 - val_loss: 8.3025 - val_accu

KeyboardInterrupt: 

In [ ]:
#model.save("models/2022_10_27__01")

: 

: 

: 

: 